In [44]:
import pandas as pd
import numpy as np
# import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [4]:
train = pd.read_csv('data/train/train.csv')
test = pd.read_csv('data/test/test.csv')
sample = pd.read_csv('data/sample_submission/sample_submission.csv')

In [5]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    if len(actual.shape)==1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val ] = 1
        actual = actual2
        
    clip = np.clip(predicted, eps, 1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [6]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [7]:
xtrain, xvalid, ytrain, yvalid  = train_test_split(train.text.values,y, stratify=y, test_size=0.1, shuffle=True)
print(xtrain.shape)
print(xvalid.shape)

(17621,)
(1958,)


## Building Basic Models
> **TF-IDF**

In [10]:
tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word', 
#                       token_pattern=r'\n{1,}',
                      ngram_range=(1,3), use_idf= True, smooth_idf=True, sublinear_tf=True, 
                      stop_words = 'english')
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

In [14]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print("logloss : %0.3f" % multiclass_logloss(yvalid, predictions))

logloss : 0.595


C:\Users\NB-0242\Anaconda3\envs\ximz\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
ctv = CountVectorizer(analyzer='word', ngram_range=(1,3), stop_words = 'english')
ctv.fit(list(xtrain) + list(xvalid))

xtrain_ctv = ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)

In [17]:
clf = LogisticRegression(C = 1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print("logloss : %0.3f" % multiclass_logloss(yvalid, predictions))

logloss : 0.554


C:\Users\NB-0242\Anaconda3\envs\ximz\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [18]:
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.470 


In [20]:
svd = decomposition.TruncatedSVD(n_components = 120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [22]:
# SVM
clf = SVC(C= 1.0, probability = True)
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print('logloss : %0.3f'% multiclass_logloss(yvalid, predictions))

logloss : 0.746


### grid search

In [23]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

In [24]:
# create pipeline
svd = TruncatedSVD()

scl = preprocessing.StandardScaler()

lr_model = LogisticRegression()


clf = pipeline.Pipeline([('svd', svd),
                        ('scl', scl),
                        ('lr', lr_model)])

In [25]:
param_grid = {'svd__n_components' : [120, 180],
                'lr__C' : [0.1, 1.0, 10],
                'lr__penalty' : ['l1', 'l2']}

In [26]:
model = GridSearchCV(estimator = clf, param_grid=param_grid, scoring = mll_scorer,
                    verbose = 10, n_jobs= -1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)
print('Best score : %0.3f' % model.best_score_)
print('Best parameters set : ')
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print('\t%s : %r' %(param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  12 out of  24 | elapsed:   15.1s remaining:   15.1s
[Parallel(n_jobs=-1)]: Done  15 out of  24 | elapsed:   16.2s remaining:    9.7s
[Parallel(n_jobs=-1)]: Done  18 out of  24 | elapsed:   19.3s remaining:    6.4s
[Parallel(n_jobs=-1)]: Done  21 out of  24 | elapsed:   21.2s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   22.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   22.0s finished
C:\Users\NB-0242\Anaconda3\envs\ximz\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Best score : -0.738
Best parameters set : 
	lr__C : 0.1
	lr__penalty : 'l2'
	svd__n_components : 180


In [27]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: -0.497
Best parameters set:
	nb__alpha: 0.1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0638s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.1s finished
C:\Users\NB-0242\Anaconda3\envs\ximz\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


### word vectors

In [39]:
embeddings_index = {}
f = open('data/glove.840B.300d.txt', encoding='utf-8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try :
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except :
        continue
f.close()

print('Found %s word' % len(embeddings_index))


0it [00:00, ?it/s]
798it [00:00, 7920.02it/s]
1649it [00:00, 8072.04it/s]
2485it [00:00, 8139.00it/s]
3322it [00:00, 8189.40it/s]
4187it [00:00, 8304.06it/s]
5041it [00:00, 8355.55it/s]
5784it [00:00, 8010.32it/s]
6625it [00:00, 8108.23it/s]
7486it [00:00, 8235.09it/s]
8340it [00:01, 8307.19it/s]
9229it [00:01, 8455.53it/s]
10076it [00:01, 8441.99it/s]
10911it [00:01, 8198.61it/s]
11769it [00:01, 8291.01it/s]
12618it [00:01, 8331.77it/s]
13491it [00:01, 8429.44it/s]
14346it [00:01, 8447.03it/s]
15197it [00:01, 8447.28it/s]
16042it [00:01, 8429.66it/s]
16894it [00:02, 8438.20it/s]
17739it [00:02, 8424.12it/s]
18582it [00:02, 8381.76it/s]
19421it [00:02, 7807.48it/s]
20273it [00:02, 7991.93it/s]
21114it [00:02, 8095.80it/s]
21953it [00:02, 8164.42it/s]
22797it [00:02, 8226.83it/s]
23640it [00:02, 8268.94it/s]
24490it [00:02, 8319.79it/s]
25347it [00:03, 8374.52it/s]
26196it [00:03, 8391.43it/s]
27039it [00:03, 8384.62it/s]
27911it [00:03, 8463.66it/s]
28758it [00:03, 8323.64it/s]
29623i

Found 2195884 word


In [49]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except : 
            continue
    M = np.array(M)
    v = M.sum(axis=0)
#     print(v.shape)
    if type(v) != np.ndarray:
        return np.zeros(300)
    division_value = np.sqrt((v **2)).sum()
#     print(division_value)
    return v / division_value

In [51]:
xtrain_glove = [sent2vec(x) for x in xtrain]
xvalid_glove = [sent2vec(x) for x in xvalid]

xtrain_glvoe = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

### DL

In [53]:
# scaling
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.fit_transform(xvalid_glove)

# label encoding
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)


In [54]:
# createlayers

model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

#compile the model
model.compile(loss=  'categorical_crossentropy', optimizer='adam')

In [56]:
model.fit(xtrain_glove_scl, y = ytrain_enc, batch_size=64, epochs = 5, verbose =1, validation_data = (xvalid_glove_scl, yvalid_enc))


Train on 17621 samples, validate on 1958 samples
Epoch 1/5
17621/17621 [==============================] - 4s 202us/step - loss: 0.9021 - val_loss: 0.7299
Epoch 2/5
17621/17621 [==============================] - 2s 116us/step - loss: 0.6935 - val_loss: 0.6981
Epoch 3/5
17621/17621 [==============================] - 2s 117us/step - loss: 0.6296 - val_loss: 0.6842
Epoch 4/5
17621/17621 [==============================] - 2s 116us/step - loss: 0.5849 - val_loss: 0.6927
Epoch 5/5
17621/17621 [==============================] - 2s 117us/step - loss: 0.5539 - val_loss: 0.6931


In [64]:
# using keras tokenizer
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))

In [67]:
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

In [68]:
# zero padding the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [77]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index)+1 , 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [83]:
word_index

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'a': 5,
 'i': 6,
 'in': 7,
 'was': 8,
 'that': 9,
 'my': 10,
 'it': 11,
 'had': 12,
 'he': 13,
 'with': 14,
 'his': 15,
 'as': 16,
 'for': 17,
 'which': 18,
 'but': 19,
 'not': 20,
 'at': 21,
 'me': 22,
 'from': 23,
 'by': 24,
 'is': 25,
 'this': 26,
 'on': 27,
 'be': 28,
 'her': 29,
 'were': 30,
 'have': 31,
 'all': 32,
 'you': 33,
 'we': 34,
 'or': 35,
 'no': 36,
 'an': 37,
 'one': 38,
 'so': 39,
 'him': 40,
 'when': 41,
 'been': 42,
 'they': 43,
 'upon': 44,
 'there': 45,
 'could': 46,
 'she': 47,
 'its': 48,
 'would': 49,
 'more': 50,
 'now': 51,
 'their': 52,
 'what': 53,
 'some': 54,
 'our': 55,
 'are': 56,
 'into': 57,
 'than': 58,
 'will': 59,
 'very': 60,
 'who': 61,
 'if': 62,
 'them': 63,
 'only': 64,
 'then': 65,
 'up': 66,
 'these': 67,
 'before': 68,
 'about': 69,
 'any': 70,
 'man': 71,
 'time': 72,
 'yet': 73,
 'out': 74,
 'said': 75,
 'even': 76,
 'did': 77,
 'your': 78,
 'might': 79,
 'after': 80,
 'old': 81,
 'like': 82,
 'f

In [85]:
embedding_matrix.shape

(25944, 300)

In [87]:
xtrain_pad.shape

(17621, 70)

In [86]:
ytrain_enc.shape

(17621, 3)

In [82]:
# a simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index)+1 , 300, weights = [embedding_matrix], input_length = max_len, trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.fit(xtrain_pad, ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 6s 348us/step - loss: 1.0779 - val_loss: 0.9808
Epoch 2/100
17621/17621 [==============================] - 5s 307us/step - loss: 0.9514 - val_loss: 0.8199
Epoch 3/100
17621/17621 [==============================] - 5s 306us/step - loss: 0.8784 - val_loss: 0.7678
Epoch 4/100
17621/17621 [==============================] - 5s 309us/step - loss: 0.8408 - val_loss: 0.7521
Epoch 5/100
17621/17621 [==============================] - 5s 308us/step - loss: 0.8126 - val_loss: 0.7394
Epoch 6/100
17621/17621 [==============================] - 5s 305us/step - loss: 0.7971 - val_loss: 0.7183
Epoch 7/100
17621/17621 [==============================] - 5s 305us/step - loss: 0.7787 - val_loss: 0.7079
Epoch 8/100
17621/17621 [==============================] - 5s 306us/step - loss: 0.7616 - val_loss: 0.7038
Epoch 9/100
17621/17621 [==============================] - 6s 314us/step - loss: 0.7433 - val_l

In [89]:
# BI directional LSTM

model = Sequential()
model.add(Embedding(len(word_index) + 1, 300 , weights = [embedding_matrix], input_length = max_len, trainable = False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 22s 1ms/step - loss: 1.0584 - val_loss: 0.8721
Epoch 2/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.8966 - val_loss: 0.7884
Epoch 3/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.8430 - val_loss: 0.7727
Epoch 4/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.8236 - val_loss: 0.7640
Epoch 5/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.7998 - val_loss: 0.7284
Epoch 6/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.7853 - val_loss: 0.7345
Epoch 7/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.7647 - val_loss: 0.7171
Epoch 8/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.7440 - val_loss: 0.6973
Epoch 9/100
17621/17621 [==============================] - 20s 1ms/step - loss: 0.7239 - val_loss: 0.67

## ENSEMBLING

In [95]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
import os
import sys
import logging

logging.basicConfig(level=logging.DEBUG, format="[%(asctime)s] %(levelname)s %(message)s",
                   datefmt = "%H:%M:%S", stream=sys.stdout)
logger = logging.getLogger(__name__)

class Ensembler(object):
    def __init__(self, model_dict, num_folds=3, task_type='classification', optimize=roc_auc_score,
                 lower_is_better=False, save_path=None):
        """
        Ensembler init function
        :param model_dict: model dictionary, see README for its format
        :param num_folds: the number of folds for ensembling
        :param task_type: classification or regression
        :param optimize: the function to optimize for, e.g. AUC, logloss, etc. Must have two arguments y_test and y_pred
        :param lower_is_better: is lower value of optimization function better or higher
        :param save_path: path to which model pickles will be dumped to along with generated predictions, or None
        """

        self.model_dict = model_dict
        self.levels = len(self.model_dict)
        self.num_folds = num_folds
        self.task_type = task_type
        self.optimize = optimize
        self.lower_is_better = lower_is_better
        self.save_path = save_path

        self.training_data = None
        self.test_data = None
        self.y = None
        self.lbl_enc = None
        self.y_enc = None
        self.train_prediction_dict = None
        self.test_prediction_dict = None
        self.num_classes = None

    def fit(self, training_data, y, lentrain):
        """
        :param training_data: training data in tabular format
        :param y: binary, multi-class or regression
        :return: chain of models to be used in prediction
        """

        self.training_data = training_data
        self.y = y

        if self.task_type == 'classification':
            self.num_classes = len(np.unique(self.y))
            logger.info("Found %d classes", self.num_classes)
            self.lbl_enc = LabelEncoder()
            self.y_enc = self.lbl_enc.fit_transform(self.y)
            kf = StratifiedKFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, self.num_classes)
        else:
            self.num_classes = -1
            self.y_enc = self.y
            kf = KFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, 1)

        self.train_prediction_dict = {}
        for level in range(self.levels):
            self.train_prediction_dict[level] = np.zeros((train_prediction_shape[0],
                                                          train_prediction_shape[1] * len(self.model_dict[level])))

        for level in range(self.levels):

            if level == 0:
                temp_train = self.training_data
            else:
                temp_train = self.train_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):
                validation_scores = []
                foldnum = 1
                for train_index, valid_index in kf.split(self.train_prediction_dict[0], self.y_enc):
                    logger.info("Training Level %d Fold # %d. Model # %d", level, foldnum, model_num)

                    if level != 0:
                        l_training_data = temp_train[train_index]
                        l_validation_data = temp_train[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])
                    else:
                        l0_training_data = temp_train[0][model_num]
                        if type(l0_training_data) == list:
                            l_training_data = [x[train_index] for x in l0_training_data]
                            l_validation_data = [x[valid_index] for x in l0_training_data]
                        else:
                            l_training_data = l0_training_data[train_index]
                            l_validation_data = l0_training_data[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])

                    logger.info("Predicting Level %d. Fold # %d. Model # %d", level, foldnum, model_num)

                    if self.task_type == 'classification':
                        temp_train_predictions = model.predict_proba(l_validation_data)
                        self.train_prediction_dict[level][valid_index,
                        (model_num * self.num_classes):(model_num * self.num_classes) +
                                                       self.num_classes] = temp_train_predictions

                    else:
                        temp_train_predictions = model.predict(l_validation_data)
                        self.train_prediction_dict[level][valid_index, model_num] = temp_train_predictions
                    validation_score = self.optimize(self.y_enc[valid_index], temp_train_predictions)
                    validation_scores.append(validation_score)
                    logger.info("Level %d. Fold # %d. Model # %d. Validation Score = %f", level, foldnum, model_num,
                                validation_score)
                    foldnum += 1
                avg_score = np.mean(validation_scores)
                std_score = np.std(validation_scores)
                logger.info("Level %d. Model # %d. Mean Score = %f. Std Dev = %f", level, model_num,
                            avg_score, std_score)

            logger.info("Saving predictions for level # %d", level)
            train_predictions_df = pd.DataFrame(self.train_prediction_dict[level])
            train_predictions_df.to_csv(os.path.join(self.save_path, "train_predictions_level_" + str(level) + ".csv"),
                                        index=False, header=None)

        return self.train_prediction_dict

    def predict(self, test_data, lentest):
        self.test_data = test_data
        if self.task_type == 'classification':
            test_prediction_shape = (lentest, self.num_classes)
        else:
            test_prediction_shape = (lentest, 1)

        self.test_prediction_dict = {}
        for level in range(self.levels):
            self.test_prediction_dict[level] = np.zeros((test_prediction_shape[0],
                                                         test_prediction_shape[1] * len(self.model_dict[level])))
        self.test_data = test_data
        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
                temp_test = self.test_data
            else:
                temp_train = self.train_prediction_dict[level - 1]
                temp_test = self.test_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):

                logger.info("Training Fulldata Level %d. Model # %d", level, model_num)
                if level == 0:
                    model.fit(temp_train[0][model_num], self.y_enc)
                else:
                    model.fit(temp_train, self.y_enc)

                logger.info("Predicting Test Level %d. Model # %d", level, model_num)

                if self.task_type == 'classification':
                    if level == 0:
                        temp_test_predictions = model.predict_proba(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict_proba(temp_test)
                    self.test_prediction_dict[level][:, (model_num * self.num_classes): (model_num * self.num_classes) +
                                                                                        self.num_classes] = temp_test_predictions

                else:
                    if level == 0:
                        temp_test_predictions = model.predict(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict(temp_test)
                    self.test_prediction_dict[level][:, model_num] = temp_test_predictions

            test_predictions_df = pd.DataFrame(self.test_prediction_dict[level])
            test_predictions_df.to_csv(os.path.join(self.save_path, "test_predictions_level_" + str(level) + ".csv"),
                                       index=False, header=None)

        return self.test_prediction_dict

In [96]:
# specify the data to be used for every level of ensembling:
train_data_dict = {0: [xtrain_tfv, xtrain_ctv, xtrain_tfv, xtrain_ctv], 1: [xtrain_glove]}
test_data_dict = {0: [xvalid_tfv, xvalid_ctv, xvalid_tfv, xvalid_ctv], 1: [xvalid_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],

              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='')

ens.fit(train_data_dict, ytrain, lentrain=xtrain_glove.shape[0])
preds = ens.predict(test_data_dict, lentest=xvalid_glove.shape[0])

NameError: name 'xgb' is not defined